In [1]:
# estimate or drop missing features
# select a date range?
# transform categorical features into str labels

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math as math
from IPython.display import display
from IPython.core.pylabtools import getfigs
from sklearn import linear_model 
import time
import pymysql as mdb
#import seaborn as sns

In [3]:
%matplotlib qt

In [4]:
#from extract_kickstarter_features import *
#%load_ext autoreload
#%autoreload 2

In [5]:
con = mdb.connect('localhost','root','hobbes','kickstarter')

In [29]:
Ids_to_analyze = []
with con:
    cur = con.cursor()
    cur.execute("DESCRIBE Predictors")
    predictor_cols = cur.fetchall()
    cur.close()

In [33]:
predictor_cols = np.array(predictor_cols)

In [36]:
np.shape(predictor_cols)

(28, 6)

In [38]:
predictor_column_names = predictor_cols[:,0]

In [40]:
print predictor_column_names

['Id' 'Url' 'Outcome' 'Pledged' 'Goal' 'Category' 'Subcategory' 'Currency'
 'Country' 'Location' 'Launch_s' 'Campaign_days' 'Title_n_words'
 'Blurb_n_words' 'Month' 'Monday_day' 'Week_day' 'Hour' 'N_rewards'
 'Min_reward' 'Median_reward' 'Max_reward' 'Reward_ship_days' 'Ships_intn'
 'N_links' 'N_figs' 'Body_n_words' 'N_creator_backed']


In [44]:
with con:
    cur = con.cursor()
    cur.execute("SELECT * FROM Predictors WHERE \
                Reward_ship_days >= 0 AND N_creator_backed >= 0 AND \
                Outcome != 'live' AND Country = 'US' AND Currency = 'USD'")
    rows = cur.fetchall()
    cur.close()

In [50]:
print np.shape(rows)

(133071, 28)


In [51]:
#predictor_df = pd.DataFrame(row)
predictor_df = pd.DataFrame(np.asarray(rows),columns=predictor_column_names)

In [52]:
predictor_df.tail()

,Id,Url,Outcome,Pledged,Goal,Category,Subcategory,Currency,Country,Location,...,N_rewards,Min_reward,Median_reward,Max_reward,Reward_ship_days,Ships_intn,N_links,N_figs,Body_n_words,N_creator_backed
133066,191380,https://www.kickstarter.com/projects/207940292...,suspended,0,2500,Installations,art/installations,USD,US,prior-lake-mn,...,1,25,25,25,92,-1,0,0,28,0
133067,191381,https://www.kickstarter.com/projects/184521459...,suspended,420,2000,Mixed Media,art/mixed media,USD,US,memphis-tn,...,14,5,40,150,41,1,18,13,750,0
133068,191382,https://www.kickstarter.com/projects/209708895...,suspended,0,31,Painting,art/painting,USD,US,lakeland-fl,...,3,8,31,1001,111,0,2,0,504,0
133069,191383,https://www.kickstarter.com/projects/thelifecu...,suspended,10084,15000,Public Art,art/public art,USD,US,black-rock-city-nv,...,5,10,200,1000,155,1,1,7,1246,130
133070,191384,https://www.kickstarter.com/projects/reaper889...,suspended,300,300000,Painting,art/painting,USD,US,douglasville-douglas-ga,...,4,300,750,5000,136,0,0,0,624,0


In [53]:
predictor_df.to_pickle('predictor_df_unprocessed.save')

In [100]:
X_cols = ['Goal','Campaign_days','Title_n_words','Blurb_n_words',
          'Month','Monday_day','Week_day','Hour','N_rewards','Min_reward','Median_reward','Max_reward',
          'Reward_ship_days','Ships_intn','N_links','N_figs','Body_n_words','N_creator_backed']

In [110]:
subset_predictor_df = predictor_df.ix[:,X_cols]

In [114]:
subset_predictor_df.head(15)

,Goal,Campaign_days,Title_n_words,Blurb_n_words,Month,Monday_day,Week_day,Hour,N_rewards,Min_reward,Median_reward,Max_reward,Reward_ship_days,Ships_intn,N_links,N_figs,Body_n_words,N_creator_backed
0,5000,30,2,17,8,12,6,20,8,10,175,5000,110,0,1,0,812,18
1,10000,30,8,21,6,30,5,14,10,5,125,5000,31,0,5,1,792,7
2,18000,27,5,14,2,25,1,8,19,5,500,10000,4,1,0,0,540,3
3,7000,56,5,17,12,6,1,17,8,10,87,1000,56,0,0,0,372,4
4,8150,30,8,16,7,22,1,11,11,4,42,815,41,-1,1,1,1405,8
5,8350,36,3,19,12,19,0,21,7,5,100,1000,103,0,5,0,760,5
6,2500,18,5,13,8,5,0,15,7,10,100,1000,26,-1,0,0,461,0
7,6000,30,6,22,5,2,2,2,12,15,80,5000,122,0,12,0,559,7
8,2500,28,5,20,10,3,0,15,7,5,75,250,28,0,0,0,426,11
9,3000,14,4,13,4,23,0,18,9,1,50,1000,68,0,8,0,764,0


In [112]:
subset_predictor_df['Ships_intn'].replace(-1,np.NaN,inplace=True)
subset_predictor_df['Ships_intn'].replace(0,-1,inplace=True)
subset_predictor_df['Ships_intn'].replace(np.NaN,0,inplace=True)

In [75]:
print np.shape(subset_predictor_df)

(133071, 21)


In [115]:
subset_predictor_df.to_pickle('subset_predictor.save')

In [87]:
outcomes_df = pd.DataFrame(0, index = np.arange(133071), columns = ['Outcome'])

In [88]:
successful_campaigns_i = np.where(predictor_df['Pledged']>=predictor_df['Goal'])[0]

In [90]:
outcomes_df.ix[successful_campaigns_i,'Outcome'] = 1

In [92]:
outcomes_df.head()

,Outcome
0,1
1,1
2,1
3,1
4,1


In [95]:
outcomes_df.to_pickle('outcomes_df.save')